In [1]:
import pandas as pd
import numpy as np
import math
from pyomo.environ import *
from pyomo.opt import SolverFactory
import os
solver_location = "C:\\Users\\DeepakYadav\\OneDrive - Tata Insights and Quants\\Optimization\\Shipment\\cbc"
os.chdir(r'C:\\Users\\DeepakYadav\\OneDrive - Tata Insights and Quants\\Optimization\\Shipment')

In [2]:
import time

In [3]:
start = time.time()

In [4]:
df = pd.read_excel('KitchTech.xlsx',sheet_name = 'Sheet1')

In [5]:
df.set_index(['Type'],inplace = True)

In [6]:
#DefiningSets
s_types = list(set(df.index))
s_i = [[1,2],[5,6]]

In [7]:
#DefiningParams
p_weight = df['Weight'].to_dict()
p_values = df['Values'].to_dict()

In [8]:
#DefiningVariables
model = ConcreteModel()

model.v_type_selected = Var(s_types, domain = Binary)
model.v_shipped = Var(s_types, domain = NonNegativeIntegers)
model.v_engage = Var(s_i, domain = Binary)

In [9]:
#DefiningConstraints
def C1_rule(model,i):
    return model.v_shipped[i] <= 10
model.C1 = Constraint(s_types, rule = C1_rule)

def C2_rule(model):
    return model.v_type_selected[4] >= model.v_type_selected[3]
model.C2 = Constraint(rule = C2_rule)

model.C6 = ConstraintList()
for i in s_i:
    model.C6.add(sum(model.v_shipped[j] for j in i) >= 4 * model.v_engage[i])
    
model.C8 = ConstraintList()
for i in s_i:
    model.C8.add(sum(model.v_shipped[j] for j in i) <= 100 * model.v_engage[i])
    
model.C7 = ConstraintList()
for i in range(len(s_i) - 1):
    model.C7.add(model.v_engage[s_i[i]] + model.v_engage[s_i[i+1]] >= 1)    

def C3_rule(model,i):
    return model.v_shipped[i] <= 1000 * model.v_type_selected[i]
model.C3 = Constraint(s_types, rule = C3_rule)

def C5_rule(model,i):
    return model.v_shipped[i] >= model.v_type_selected[i]
model.C5 = Constraint(s_types,rule = C5_rule)

def C4_rule(model):
    return sum(model.v_shipped[i] * p_weight[i] for i in s_types) <= 1000
model.C4 = Constraint(rule = C4_rule)

In [10]:
#DefiningObjective
model.value = Objective(
    expr=sum(model.v_shipped[i] * p_values[i] for i in s_types),
    sense=maximize
)

In [11]:
#Run Model
optimizer = SolverFactory("cbc", executable = solver_location)
#optimizer.options['ratio'] = 7e-2
results = optimizer.solve(model,tee=True) 
results.write()
print("Optimal Solution:")

Welcome to the CBC MILP Solver 
Version: 2.10 
Build Date: Oct 13 2019 

command line - C:\Users\DeepakYadav\OneDrive - Tata Insights and Quants\Optimization\Shipment\cbc.exe -printingOptions all -import C:\Users\DEEPAK~1\AppData\Local\Temp\tmp7ifc8sfq.pyomo.lp -stat=1 -solve -solu C:\Users\DEEPAK~1\AppData\Local\Temp\tmp7ifc8sfq.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
 CoinLpIO::readLp(): Maximization problem reformulated as minimization
Coin0009I Switching back to maximization to get correct duals etc
Presolve is modifying 6 integer bounds and re-presolving
Presolve 19 (-6) rows, 14 (0) columns and 46 (-6) elements
Statistics for presolved model
Original problem has 14 integers (8 of which binary)
==== 8 zero objective 6 different
8 variables have objective of -0
1 variables have objective of 20
1 variables have objective of 40
1 variables have objective of 60
2 variables have objective of 70
1 variables have objective of 90
==== absolute

In [26]:
#Post Process
for s in s_types:
    if model.v_shipped[s].value>0:
            print(f" type {s}: # of shipped is {model.v_shipped[s].value}")

 type 1: # of shipped is 10.0
 type 2: # of shipped is 10.0
 type 6: # of shipped is 7.0


In [13]:
end = time.time()

In [14]:
elapsed_time = end - start
elapsed_time

0.8431961536407471

In [15]:
model.v_shipped.pprint()

v_shipped : Size=6, Index=v_shipped_index
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      1 :     0 :  10.0 :  None : False : False : NonNegativeIntegers
      2 :     0 :  10.0 :  None : False : False : NonNegativeIntegers
      3 :     0 :   0.0 :  None : False : False : NonNegativeIntegers
      4 :     0 :   0.0 :  None : False : False : NonNegativeIntegers
      5 :     0 :   0.0 :  None : False : False : NonNegativeIntegers
      6 :     0 :   7.0 :  None : False : False : NonNegativeIntegers


In [16]:
model.v_type_selected.pprint()

v_type_selected : Size=6, Index=v_type_selected_index
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      1 :     0 :   1.0 :     1 : False : False : Binary
      2 :     0 :   1.0 :     1 : False : False : Binary
      3 :     0 :   0.0 :     1 : False : False : Binary
      4 :     0 :   0.0 :     1 : False : False : Binary
      5 :     0 :   0.0 :     1 : False : False : Binary
      6 :     0 :   1.0 :     1 : False : False : Binary


In [17]:
model.C2.pprint()

C2 : Size=1, Index=None, Active=True
    Key  : Lower : Body                                    : Upper : Active
    None :  -Inf : v_type_selected[3] - v_type_selected[4] :   0.0 :   True


In [18]:
model.C1.pprint()

C1 : Size=6, Index=C1_index, Active=True
    Key : Lower : Body         : Upper : Active
      1 :  -Inf : v_shipped[1] :  10.0 :   True
      2 :  -Inf : v_shipped[2] :  10.0 :   True
      3 :  -Inf : v_shipped[3] :  10.0 :   True
      4 :  -Inf : v_shipped[4] :  10.0 :   True
      5 :  -Inf : v_shipped[5] :  10.0 :   True
      6 :  -Inf : v_shipped[6] :  10.0 :   True


In [19]:
model.v_engage.pprint()

v_engage : Size=2, Index=v_engage_index
    Key    : Lower : Value : Upper : Fixed : Stale : Domain
    (1, 2) :     0 :   1.0 :     1 : False : False : Binary
    (5, 6) :     0 :   1.0 :     1 : False : False : Binary


In [20]:
model.C7.pprint()

C7 : Size=1, Index=C7_index, Active=True
    Key : Lower : Body                          : Upper : Active
      1 :   1.0 : v_engage[1,2] + v_engage[5,6] :  +Inf :   True


In [21]:
model.C6.pprint()

C6 : Size=2, Index=C6_index, Active=True
    Key : Lower : Body                                            : Upper : Active
      1 :  -Inf : 4*v_engage[1,2] - (v_shipped[1] + v_shipped[2]) :   0.0 :   True
      2 :  -Inf : 4*v_engage[5,6] - (v_shipped[5] + v_shipped[6]) :   0.0 :   True


# Comments

In [22]:
# model.C20 = ConstraintList()
# for i in s_i:
#     g = sum(model.v_shipped[j] for j in i)
#     model.C20.add(g - 4 <= 1000 * model.v_engage[i])

In [23]:
# v = [1,2]
# def C6_rule(model):
#     return sum(model.v_shipped[i] for i in v) >=  4
# model.C6 = Constraint(rule = C6_rule)

# u = [5,6]
# def C7_rule(model):
#     return sum(model.v_shipped[i] for i in u) >= 4
# model.C7 = Constraint(rule = C7_rule)

In [24]:
# model.C4 = Constraint(expr=sum(model.v_shipped[i] * p_weight[i] for i in s_types) <= 1000)

In [25]:
# #Post Process
# df_out=pd.DataFrame()
# for s in s_types:
#     df_out=df_out.append([[s,model.v_shipped[s].value]],ignore_index=True)
#     if model.v_shipped[s].value>0:
#             print(f" type {s}: # of shipped is {model.v_shipped[s].value}")
#             #df_out=df_out.append([[i,j,model.v_x[i,j].value]],ignore_index=True)
# df_out.columns=['Types' ,'Values']
# df_out=pd.pivot(df_out,columns ='Types',values='Values')
# df_out.to_excel('KitchTech Shipment.xlsx')